<a href="https://colab.research.google.com/github/ichekhovskikh/skip-thought-vectors/blob/master/skip_thoughts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!echo -e "\n[global]\nfloatX=float32\n" >> ~/.theanorc

import importlib.util
import sys
from google.colab import drive

sys.path.append('/content/drive/My Drive/skip-thoughts-master/training')
import vocab
import train
import tools

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
text = ["первое предложение", "второе предложение", "третье предложение", "что-то еще предложение"]
worddict, wordcount = vocab.build_dictionary(text)
vocab.save_dictionary(worddict, wordcount, "/content/drive/My Drive/sample_data/vocab.pkl")

train.trainer(
    text, 
    n_words=len(worddict) + 2,
    dictionary="/content/drive/My Drive/sample_data/vocab.pkl",
    saveto="/content/drive/My Drive/sample_data/model.npz",
    saveFreq=1)



{'dim_word': 620, 'dim': 2400, 'encoder': 'gru', 'decoder': 'gru', 'max_epochs': 5, 'dispFreq': 1, 'decay_c': 0.0, 'grad_clip': 5.0, 'n_words': 8, 'maxlen_w': 30, 'optimizer': 'adam', 'batch_size': 64, 'saveto': '/content/drive/My Drive/sample_data/model.npz', 'dictionary': '/content/drive/My Drive/sample_data/vocab.pkl', 'saveFreq': 1, 'reload_': False}
Loading dictionary...
Building model
Building f_log_probs...


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Done
Building f_cost...
Done
Done
Building f_grad...
Building optimizers...
Optimization
Epoch  0
Epoch  0 Update  1 Cost  26.77531 UD  2.6113743782043457
Saving...
Done
Seen 2 samples
Epoch  1
Epoch  1 Update  2 Cost  12.288796 UD  2.6033074855804443
Saving...
Done
Seen 2 samples
Epoch  2
Epoch  2 Update  3 Cost  4.2547326 UD  2.4849472045898438
Saving...
Done
Seen 2 samples
Epoch  3
Epoch  3 Update  4 Cost  0.56317544 UD  2.474827527999878
Saving...
Done
Seen 2 samples
Epoch  4
Epoch  4 Update  5 Cost  0.014670965 UD  2.4784438610076904
Saving...
Done
Seen 2 samples
Finish


In [5]:
from gensim.models import Word2Vec, KeyedVectors

text_split_on_words = [['первое', 'предложение'], ['второе', 'предложение'], ['третье', 'предложение'], ['что-то', 'еще', 'предложение']]

model = Word2Vec(text_split_on_words, size=300, window=5, min_count=1, workers=4)
model.wv.save_word2vec_format('/content/drive/My Drive/sample_data/word2vec.bin', binary=True)
embed_map = KeyedVectors.load_word2vec_format('/content/drive/My Drive/sample_data/word2vec.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [6]:
model = tools.load_model(
    path_to_model = '/content/drive/My Drive/sample_data/model.npz',
    path_to_dictionary = '/content/drive/My Drive/sample_data/vocab.pkl',
    path_to_word2vec = '/content/drive/My Drive/sample_data/word2vec.bin')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Loading dictionary...
Creating inverted dictionary...
Loading model options...
Loading model parameters...
Compiling encoder...
Creating word lookup tables...
Packing up...


In [0]:
vectors = tools.encode(model, text)

In [0]:
def cosine_similarity(first_vector, second_vector):
    numerator = sum([x * y for x, y in zip(first_vector, second_vector)])
    first_norm = pow(sum([x * x for x in first_vector]), 0.5)
    second_norm = pow(sum([x * x for x in second_vector]), 0.5)
    return numerator / (first_norm * second_norm)

In [23]:
print(cosine_similarity(vectors[0], vectors[1]))

0.45222118707126796
